# CH08-01: Hugging Face 生態簡介

**課程**: iSpan Python NLP Cookbooks v2
**章節**: CH08 Hugging Face 實戰
**版本**: v1.0
**更新日期**: 2025-10-17

---

## 📚 本節學習目標

1. 理解 Hugging Face 生態系統的核心組件
2. 掌握 Transformers 函式庫的基本架構
3. 熟悉 Hugging Face Hub 模型與數據集資源
4. 了解模型卡片 (Model Card) 的重要性
5. 學會快速搜尋與選擇預訓練模型

---

## 1. Hugging Face 生態系統概覽

### 1.1 什麼是 Hugging Face?

**Hugging Face** 是目前最流行的 NLP 開源社群平台,提供:

- 🤗 **Transformers**: 預訓練模型函式庫 (PyTorch, TensorFlow, JAX)
- 🗂️ **Datasets**: 數據集函式庫 (超過 50,000 個數據集)
- 🏛️ **Hub**: 模型與數據集共享平台 (超過 500,000 個模型)
- ⚡ **Accelerate**: 分散式訓練加速工具
- 🔍 **Tokenizers**: 高效能分詞器

**核心優勢**:
- ✅ 統一 API,支援所有主流模型 (BERT, GPT, T5...)
- ✅ 開箱即用,無需從零訓練
- ✅ 活躍社群,持續更新最新模型
- ✅ 完整文檔與教學資源

In [ ]:
# 安裝 Hugging Face 核心套件
# !pip install transformers datasets accelerate tokenizers -q

# 驗證安裝
import transformers
import datasets

print(f"✅ Transformers 版本: {transformers.__version__}")
print(f"✅ Datasets 版本: {datasets.__version__}")

### 1.2 生態系統架構圖

```
┌─────────────────────────────────────────────┐
│           Hugging Face 生態系統              │
├─────────────────────────────────────────────┤
│                                             │
│  ┌──────────┐  ┌──────────┐  ┌──────────┐ │
│  │   Hub    │  │Transformers│ │ Datasets │ │
│  │ (模型庫) │  │ (模型API)  │ │ (數據集) │ │
│  └──────────┘  └──────────┘  └──────────┘ │
│       ↓              ↓              ↓      │
│  ┌──────────┐  ┌──────────┐  ┌──────────┐ │
│  │Tokenizers│  │Accelerate│  │ Evaluate │ │
│  │ (分詞器) │  │ (加速器) │  │ (評估)   │ │
│  └──────────┘  └──────────┘  └──────────┘ │
└─────────────────────────────────────────────┘
```

---

## 2. Transformers 函式庫核心概念

### 2.1 三大核心組件

Transformers 函式庫圍繞三個核心概念:

1. **Model (模型)**: 預訓練的神經網路
2. **Tokenizer (分詞器)**: 文本 → 數字的轉換器
3. **Pipeline (管道)**: 端到端的快速應用 API

#### 2.1.1 Model (模型)

**模型架構分類**:

In [ ]:
from transformers import AutoModel, AutoModelForSequenceClassification

# 載入預訓練模型
model_name = "distilbert-base-uncased"

# 1. 基礎模型 (無任務頭)
base_model = AutoModel.from_pretrained(model_name)
print(f"基礎模型: {base_model.__class__.__name__}")
print(f"參數量: {base_model.num_parameters():,}")

# 2. 任務特定模型 (有分類頭)
classifier_model = AutoModelForSequenceClassification.from_pretrained(
    model_name, 
    num_labels=2  # 二分類任務
)
print(f"\n分類模型: {classifier_model.__class__.__name__}")
print(f"參數量: {classifier_model.num_parameters():,}")

**模型命名規範**:

| 前綴 | 說明 | 範例 |
|------|------|------|
| `AutoModel` | 基礎模型 (無任務頭) | `AutoModel.from_pretrained('bert-base')` |
| `AutoModelForSequenceClassification` | 序列分類 | 情感分析、文本分類 |
| `AutoModelForTokenClassification` | 標記分類 | 命名實體識別 (NER) |
| `AutoModelForQuestionAnswering` | 問答系統 | SQuAD, DRCD |
| `AutoModelForCausalLM` | 因果語言模型 | GPT, LLaMA (文本生成) |
| `AutoModelForSeq2SeqLM` | 序列到序列 | T5, BART (翻譯、摘要) |

#### 2.1.2 Tokenizer (分詞器)

In [ ]:
from transformers import AutoTokenizer

# 載入分詞器 (必須與模型匹配)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# 文本編碼
text = "Hugging Face is amazing!"
encoded = tokenizer(text, return_tensors="pt")

print("原始文本:", text)
print("\n編碼結果:")
print(f"input_ids: {encoded['input_ids']}")
print(f"attention_mask: {encoded['attention_mask']}")

# 解碼回文本
decoded = tokenizer.decode(encoded['input_ids'][0])
print(f"\n解碼文本: {decoded}")

# 查看詞彙表大小
print(f"\n詞彙表大小: {tokenizer.vocab_size:,}")

**Tokenizer 重要參數**:

| 參數 | 說明 | 範例 |
|------|------|------|
| `return_tensors` | 返回張量類型 | `"pt"` (PyTorch), `"tf"` (TensorFlow) |
| `padding` | 填充策略 | `True`, `"max_length"`, `"longest"` |
| `truncation` | 截斷策略 | `True`, `"only_first"`, `"longest_first"` |
| `max_length` | 最大序列長度 | `512`, `128` |
| `add_special_tokens` | 是否添加特殊標記 | `True` (默認) |

#### 2.1.3 Pipeline (管道)

**Pipeline 是最快速的使用方式**,封裝了模型、分詞器、後處理邏輯:

In [ ]:
from transformers import pipeline

# 1. 情感分析 Pipeline
sentiment_pipeline = pipeline("sentiment-analysis")
result = sentiment_pipeline("I love this tutorial!")
print("情感分析:", result)

# 2. 命名實體識別 Pipeline
ner_pipeline = pipeline("ner", aggregation_strategy="simple")
result = ner_pipeline("Hugging Face is based in New York City.")
print("\n命名實體識別:", result)

# 3. 文本生成 Pipeline
generator = pipeline("text-generation", model="gpt2")
result = generator(
    "Once upon a time",
    max_length=50,
    num_return_sequences=1
)
print("\n文本生成:", result[0]['generated_text'])

**內建 Pipeline 任務列表**:

| Pipeline 名稱 | 任務 | 範例應用 |
|--------------|------|----------|
| `sentiment-analysis` | 情感分析 | 評論正負面判斷 |
| `ner` | 命名實體識別 | 抽取人名、地名、組織 |
| `question-answering` | 問答系統 | 從文本中找答案 |
| `text-generation` | 文本生成 | 自動寫作、續寫 |
| `summarization` | 文本摘要 | 新聞摘要、論文總結 |
| `translation` | 機器翻譯 | 多語言翻譯 |
| `zero-shot-classification` | 零樣本分類 | 無需訓練的分類 |
| `fill-mask` | 完形填空 | BERT 式填空任務 |

---

## 3. Hugging Face Hub 模型資源

### 3.1 瀏覽與搜尋模型

In [ ]:
from huggingface_hub import list_models

# 搜尋情感分析模型 (限制前 5 個)
models = list_models(
    filter="text-classification",
    sort="downloads",
    direction=-1,
    limit=5
)

print("🔝 最受歡迎的情感分析模型:\n")
for i, model in enumerate(models, 1):
    print(f"{i}. {model.modelId}")
    print(f"   下載次數: {model.downloads:,}")
    print(f"   標籤: {model.tags[:5]}\n")

### 3.2 模型卡片 (Model Card) 解讀

**模型卡片包含的關鍵信息**:

1. **Model Description**: 模型簡介與架構
2. **Intended Use**: 預期用途與限制
3. **Training Data**: 訓練數據來源
4. **Training Procedure**: 訓練細節 (超參數、硬體)
5. **Evaluation Results**: 評估指標與基準對比
6. **Limitations**: 已知限制與偏差
7. **How to Use**: 使用範例代碼

**範例: 查看模型卡片**

In [ ]:
from huggingface_hub import ModelCard

# 載入模型卡片
model_id = "distilbert-base-uncased-finetuned-sst-2-english"
card = ModelCard.load(model_id)

# 顯示模型卡片部分內容
print(f"📄 模型: {model_id}")
print(f"\n模型簡介:\n{card.text[:500]}...")  # 顯示前 500 字元

### 3.3 模型選擇決策樹

```
選擇 Hugging Face 模型的流程:

1. 確定任務類型
   ├─ 分類 → text-classification
   ├─ NER → token-classification
   ├─ 生成 → text-generation
   └─ 問答 → question-answering

2. 選擇模型大小
   ├─ 資源受限 → distilbert, albert, mobile
   ├─ 平衡效能 → bert-base, roberta-base
   └─ 極致效能 → bert-large, roberta-large

3. 考慮語言
   ├─ 英文 → bert, roberta, gpt2
   ├─ 中文 → bert-base-chinese, roberta-wwm-ext
   └─ 多語言 → mbert, xlm-roberta

4. 檢查 Fine-tune 狀態
   ├─ 已微調 (task-specific) → 直接使用
   └─ 預訓練 (pretrained) → 需自行微調
```

---

## 4. Datasets 函式庫

### 4.1 載入內建數據集

In [ ]:
from datasets import load_dataset

# 載入 IMDB 電影評論數據集
dataset = load_dataset("imdb", split="train[:100]")  # 先載入 100 筆測試

print(f"數據集大小: {len(dataset)}")
print(f"\n欄位: {dataset.column_names}")
print(f"\n第一筆資料:")
print(dataset[0])

### 4.2 數據集基本操作

In [ ]:
# 過濾數據
positive_reviews = dataset.filter(lambda x: x['label'] == 1)
print(f"正面評論數量: {len(positive_reviews)}")

# 映射函數 (添加文本長度欄位)
dataset_with_length = dataset.map(
    lambda x: {"text_length": len(x['text'])}
)
print(f"\n新增欄位: {dataset_with_length.column_names}")

# 查看統計信息
import numpy as np
lengths = dataset_with_length['text_length']
print(f"\n文本長度統計:")
print(f"  平均: {np.mean(lengths):.0f} 字元")
print(f"  最大: {max(lengths)} 字元")
print(f"  最小: {min(lengths)} 字元")

### 4.3 數據集格式轉換

In [ ]:
# 轉換為 Pandas DataFrame
df = dataset.to_pandas()
print("Pandas DataFrame:")
print(df.head(3))

# 轉換為 PyTorch Dataset
dataset.set_format(type="torch", columns=["label"])
print(f"\nPyTorch 格式: {dataset[0]}")

---

## 5. 實戰案例: 完整流程示範

### 5.1 情感分析完整流程

In [ ]:
from transformers import pipeline

# Step 1: 載入預訓練模型 (Pipeline 封裝)
classifier = pipeline(
    "sentiment-analysis",
    model="distilbert-base-uncased-finetuned-sst-2-english"
)

# Step 2: 準備測試數據
test_texts = [
    "This movie is absolutely fantastic!",
    "I hated every minute of it.",
    "It was okay, nothing special.",
    "Best film I've seen this year!"
]

# Step 3: 批次預測
results = classifier(test_texts)

# Step 4: 顯示結果
for text, result in zip(test_texts, results):
    label = result['label']
    score = result['score']
    print(f"文本: {text}")
    print(f"預測: {label} (信心度: {score:.2%})\n")

### 5.2 手動流程 (不使用 Pipeline)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Step 1: 載入模型與分詞器
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Step 2: 編碼文本
text = "Hugging Face makes NLP so easy!"
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

# Step 3: 模型推理
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predictions = torch.softmax(logits, dim=-1)

# Step 4: 解析結果
predicted_class = torch.argmax(predictions, dim=-1).item()
confidence = predictions[0][predicted_class].item()

label_map = {0: "NEGATIVE", 1: "POSITIVE"}
print(f"文本: {text}")
print(f"預測: {label_map[predicted_class]}")
print(f"信心度: {confidence:.2%}")
print(f"\n原始 logits: {logits}")
print(f"softmax 機率: {predictions}")

---

## 6. 最佳實踐與常見問題

### 6.1 最佳實踐

1. **選擇合適的模型大小**:
   - 原型開發: `distilbert`, `albert-base`
   - 生產部署: `bert-base`, `roberta-base`
   - 學術研究: `bert-large`, `roberta-large`

2. **使用 `AutoModel` 而非具體模型類**:
   ```python
   # ✅ 推薦
   from transformers import AutoModel
   model = AutoModel.from_pretrained("bert-base-uncased")
   
   # ❌ 不推薦
   from transformers import BertModel
   model = BertModel.from_pretrained("bert-base-uncased")
   ```

3. **快取模型以加速載入**:
   ```python
   # 模型會自動快取到 ~/.cache/huggingface/
   # 第二次載入時會直接從本地讀取
   ```

4. **使用 `device_map` 進行多 GPU 推理**:
   ```python
   model = AutoModel.from_pretrained(
       "bert-large-uncased",
       device_map="auto"  # 自動分配到可用 GPU
   )
   ```

### 6.2 常見問題

**Q1: 模型下載失敗怎麼辦?**
```python
# 使用鏡像站 (中國大陸用戶)
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
```

**Q2: 記憶體不足 (OOM)?**
```python
# 1. 使用更小的模型
# 2. 減少 batch size
# 3. 使用量化模型
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(
    "gpt2",
    load_in_8bit=True  # 8-bit 量化
)
```

**Q3: 如何離線使用模型?**
```python
# 預先下載模型
model = AutoModel.from_pretrained("bert-base-uncased")
model.save_pretrained("./local_model")

# 離線載入
model = AutoModel.from_pretrained("./local_model")
```

---

## 7. 課後練習

### 練習 1: 探索不同任務的 Pipeline

嘗試使用以下 Pipeline:
1. `fill-mask`: 完形填空
2. `question-answering`: 問答系統
3. `summarization`: 文本摘要

In [ ]:
# 練習 1: Fill-Mask
from transformers import pipeline

unmasker = pipeline("fill-mask", model="bert-base-uncased")
result = unmasker("Hugging Face is [MASK] for NLP.")
print("Fill-Mask 結果:")
for r in result[:3]:
    print(f"  {r['sequence']} (score: {r['score']:.2%})")

In [ ]:
# 練習 2: Question Answering
qa_pipeline = pipeline("question-answering")

context = """
Hugging Face is a company based in New York City. 
It was founded in 2016 and specializes in natural language processing.
"""

question = "When was Hugging Face founded?"
result = qa_pipeline(question=question, context=context)

print(f"問題: {question}")
print(f"答案: {result['answer']} (信心度: {result['score']:.2%})")

### 練習 2: 比較不同模型效能

比較 `distilbert-base-uncased` 與 `bert-base-uncased` 的參數量與推理速度:

In [ ]:
import time
from transformers import AutoModel, AutoTokenizer

models_to_compare = [
    "distilbert-base-uncased",
    "bert-base-uncased"
]

test_text = "Comparing model performance" * 10  # 重複 10 次

for model_name in models_to_compare:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    
    inputs = tokenizer(test_text, return_tensors="pt")
    
    # 測量推理時間
    start = time.time()
    with torch.no_grad():
        _ = model(**inputs)
    elapsed = time.time() - start
    
    print(f"\n模型: {model_name}")
    print(f"  參數量: {model.num_parameters():,}")
    print(f"  推理時間: {elapsed*1000:.2f} ms")

---

## 8. 本節總結

### ✅ 關鍵要點

1. **Hugging Face 生態系統**:
   - Transformers (模型), Datasets (數據), Hub (平台)
   - 統一 API,支援所有主流模型

2. **三大核心組件**:
   - Model: 預訓練神經網路
   - Tokenizer: 文本編碼/解碼
   - Pipeline: 端到端快速應用

3. **模型選擇策略**:
   - 任務類型 → 模型架構
   - 資源限制 → 模型大小
   - 語言需求 → 預訓練語料

4. **最佳實踐**:
   - 使用 `AutoModel` 提升彈性
   - 利用快取機制加速開發
   - 閱讀模型卡片了解限制

### 📚 延伸閱讀

- [Hugging Face 官方文檔](https://huggingface.co/docs/transformers)
- [Hugging Face 課程](https://huggingface.co/course)
- [模型 Hub](https://huggingface.co/models)

### 🚀 下一節預告

**CH08-02: Pipeline API 快速入門**
- 深入 Pipeline 內部機制
- 自訂 Pipeline 參數
- 批次處理與效能優化

---

**課程**: iSpan Python NLP Cookbooks v2
**講師**: Claude AI
**最後更新**: 2025-10-17